In [2]:
import nibabel as nib
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Conv3D, MaxPooling3D, Conv3DTranspose, concatenate
import tensorflow as tf
from tensorflow.keras import backend as K
import matplotlib.pyplot as plt
import numpy as np
from skimage.transform import resize
import skimage
import keras

In [6]:
for_file = '32256256'
for_model = (32 ,256 ,256, 1)
for_processing = (32 ,256 ,256)

In [10]:
image_path = r"C:\Users\mh490\Desktop\Semester 8\FYP - 2\FYP WEB FINAL VERSION\Test Images\Img_01.nii"

In [11]:
def load_nifti_image(image_path, target_shape=for_processing):
    image = nib.load(image_path).get_fdata(dtype=np.float32)
    image = image / np.max(image)
    image_resized = resize(image, target_shape, mode='constant', preserve_range=True)
    image_resized = image_resized[..., np.newaxis]
    return image_resized

In [12]:
image = load_nifti_image(image_path)

In [13]:
def conv_block(input_tensor, num_filters):
    x = Conv3D(num_filters, (3, 3, 3), activation='relu', padding='same')(input_tensor)
    x = Conv3D(num_filters, (3, 3, 3), activation='relu', padding='same')(x)
    return x

def encoder_block(input_tensor, num_filters):
    x = conv_block(input_tensor, num_filters)
    p = MaxPooling3D((2, 2, 2))(x)
    return x, p

def decoder_block(input_tensor, concat_tensor, num_filters):
    x = Conv3DTranspose(num_filters, (2, 2, 2), strides=(2, 2, 2), padding='same')(input_tensor)
    x = concatenate([x, concat_tensor], axis=-1)
    x = conv_block(x, num_filters)
    return x

def build_unet(input_shape, n_classes):
    inputs = Input(input_shape)

    # Encoder
    s1, p1 = encoder_block(inputs, 64)
    s2, p2 = encoder_block(p1, 128)
    s3, p3 = encoder_block(p2, 256)
    #s4, p4 = encoder_block(p3, 512)

    # Bridge
    b1 = conv_block(p3, 1024)

    # Decoder
    d1 = decoder_block(b1, s3, 256)
    d2 = decoder_block(d1, s2, 128)
    d3 = decoder_block(d2, s1, 64)
    #d4 = decoder_block(d3, s1, 64)

    # Output layer
    if n_classes == 1:
        activation = 'sigmoid'
    else:
        activation = 'softmax'
    outputs = Conv3D(n_classes, (1, 1, 1), padding="same", activation=activation)(d3)

    model = Model(inputs=[inputs], outputs=[outputs], name="3D-U-Net")

    return model

def dice_coefficient(y_true, y_pred, smooth=1e-6):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def dice_loss(y_true, y_pred):
    return 1 - dice_coefficient(y_true, y_pred)

def bce_dice_loss(y_true, y_pred):
    return tf.keras.losses.binary_crossentropy(y_true, y_pred) + dice_loss(y_true, y_pred)

# Create the model
input_shape = for_model
n_classes = 1
model = build_unet(input_shape, n_classes)

model.compile(optimizer='adam', loss=bce_dice_loss, metrics=[dice_coefficient])

model.summary()

Model: "3D-U-Net"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)      │ (None, 32, 256, 256, 1)   │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv3d (Conv3D)               │ (None, 32, 256, 256, 64)  │           1,792 │ input_layer[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv3d_1 (Conv3D)             │ (None, 32, 256, 256, 64)  │         110,656 │ conv3d[0][0]               │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ max_pooling3d (MaxPooling3D)  │ (None, 16, 128, 128, 64)  │               0 │ conv3d_1[0][0]             │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv3d_2 (Conv3D)             │ (None, 16, 128, 128, 128) │         221,312 │ max_pooling3d[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv3d_3 (Conv3D)             │ (None, 16, 128, 128, 128) │         442,496 │ conv3d_2[0][0]             │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ max_pooling3d_1               │ (None, 8, 64, 64, 128)    │               0 │ conv3d_3[0][0]             │
│ (MaxPooling3D)                │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv3d_4 (Conv3D)             │ (None, 8, 64, 64, 256)    │         884,992 │ max_pooling3d_1[0][0]      │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv3d_5 (Conv3D)             │ (None, 8, 64, 64, 256)    │       1,769,728 │ conv3d_4[0][0]             │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ max_pooling3d_2               │ (None, 4, 32, 32, 256)    │               0 │ conv3d_5[0][0]             │
│ (MaxPooling3D)                │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv3d_6 (Conv3D)             │ (None, 4, 32, 32, 1024)   │       7,078,912 │ max_pooling3d_2[0][0]      │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv3d_7 (Conv3D)             │ (None, 4, 32, 32, 1024)   │      28,312,576 │ conv3d_6[0][0]             │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv3d_transpose              │ (None, 8, 64, 64, 256)    │       2,097,408 │ conv3d_7[0][0]             │
│ (Conv3DTranspose)             │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ concatenate (Concatenate)     │ (None, 8, 64, 64, 512)    │               0 │ conv3d_transpose[0][0],    │
│                               │                           │                 │ conv3d_5[0][0]             │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv3d_8 (Conv3D)             │ (None, 8, 64, 64, 256)    │       3,539,200 │ concatenate[0][0]          │
├───────────────────────────────┼───────────────────────────┼───────────────

 Total params: 48,216,001 (183.93 MB)

 Trainable params: 48,216,001 (183.93 MB)

 Non-trainable params: 0 (0.00 B)

In [14]:
file_path = r"C:\Users\mh490\Desktop\Semester 8\FYP - 2\FYP WEB FINAL VERSION\Models\model_checkpoint_256_100_epochs_full_unet256_1024_3_layers.keras"
model.load_weights(file_path)

C:\Users\mh490\anaconda3\Lib\site-packages\keras\src\saving\saving_lib.py:396: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 74 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


In [15]:
image.shape

(32, 256, 256, 1)

In [16]:
image = np.expand_dims(image, axis=0) 

In [17]:
image.shape

(1, 32, 256, 256, 1)

In [18]:
image[0]

array([[[[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        ...,

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]]],


       [[[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        ...,

        [[0.],
 

In [19]:
test_predictions = model.predict(image)

1/1 ━━━━━━━━━━━━━━━━━━━━ 15s 15s/step


In [21]:
test_predictions.shape

(1, 32, 256, 256, 1)